In [43]:
using Plots
using LaTeXStrings
using GLMakie
using Distributions, Random
# using ProfileVega
# using Traceur

# Parameters

In [44]:
# set_zero_subnormals(true)

In [45]:
const global U0 = 10;
const global R = 0.5;
const global Rtr = 1.5 * R

const global xmax = 2;
const global xmin = -2;
const global ymax = 2;
const global ymin = -2;
const global G = [0, -9.81];
const global k_b = 1.3806490e-23;
# FLUID
global μ = 1.813e-5
global ρ = 1.2

1.2

In [46]:
function Stokes(d)
    tchar = ρ_d * (d^2) / (18 * μ)
    return (U0 / R) * tchar
end

function invStokes(St)
    return sqrt(18 * St * μ * R / (U0 * ρ_d))
end

invStokes (generic function with 1 method)

### Functions 

In [47]:
function RungeKutta(fun, t0, tn, dt, y0, checkFun)
    y = []
    push!(y, y0)

    steps = round(Int, (tn - t0) / dt)
    for i in 1:steps
        k1 = dt .* fun(checkFun(y[i]))
        k2 = dt .* fun(checkFun(y[i] .+ k1 ./ 2))
        k3 = dt .* fun(checkFun(y[i] .+ k2 ./ 2))
        k4 = dt .* fun(checkFun(y[i] .+ k3))
        k = (k1 + 2 * k2 + 2 * k3 + k4) / 6
        push!(y, checkFun(y[i] + k))
    end
    return y
end

function RK4systems(f, g, t0, tn, dt, y0, V0, checkFuns)
    #    RK4systems(dxdtP, dvdtP, t0, tend, dt, points, V0);
    x = Vector{Matrix{Float64}}(undef, 0)
    v = Vector{Matrix{Float64}}(undef, 0)


    push!(x, y0)
    push!(v, V0)
    steps = round(Int, (tn - t0) / dt)
    for i in 1:steps
        k1 = zeros(size(x[i]))
        k2 = zeros(size(x[i]))
        k3 = zeros(size(x[i]))
        k4 = zeros(size(x[i]))

        l1 = zeros(size(v[i]))
        l2 = zeros(size(v[i]))
        l3 = zeros(size(v[i]))
        l4 = zeros(size(v[i]))

        f(k1, x[i], v[i])
        g(l1, x[i], v[i])

        f(k2, x[i] .+ (k1 ./ 2), v[i] .+ l1 ./ 2)
        g(l2, x[i] .+ (k1 ./ 2), v[i] .+ l1 ./ 2)

        f(k3, x[i] .+ (k2 ./ 2), v[i] .+ l2 ./ 2)
        g(l3, x[i] .+ (k2 ./ 2), v[i] .+ l2 ./ 2)

        f(k4, x[i] .+ k3, v[i] .+ l3)
        g(l4, x[i] .+ k3, v[i] .+ l3)

        k = (k1 .+ 2 .* k2 .+ 2 .* k3 .+ k4) ./ 6
        l = (l1 .+ 2 .* l2 .+ 2 .* l3 .+ l4) ./ 6

        newX = x[i] + k
        newV =  v[i] + l
        
        for chkFun in checkFuns
            newX, newV = chkFun(newX,newV)
        end

        push!(x, newX)
        push!(v, newV)

        if any(isnan, x[i])
            println("Blew UP at $i")
            break
        elseif size(x[i+1])[2] == 0
            break
        end
    end
    return x
end

function meshgrid(x, y)
    X = [i for i in x, j in 1:length(y)]
    Y = [j for i in 1:length(x), j in y]

    return cat(X, Y, dims=3)
end

function Pressure(coord, vfun)
    U = vfun(points)
    @views u = U[1, :]
    @views v = U[2, :]
    return @. 0.5ρ * U0^2 - 0.5ρ * (u^2 + v^2)
end

function isoPsi(C, tol, zs, xs, ys, N)
    sols = []
    for i in 1:N
        for j in 1:N
            if abs(zs[i, j] - C) < tol
                push!(sols, [xs[i], ys[j]])
            end
        end
    end
    return sols
end

function dxdtP!(DX, points, V)
    DX[:] = dt .* V
    nothing
end

function Knudsen(coord, T_f)
    c = @. sqrt(1.4 * 287.14 * T_f)
    Kn = @. μ / (D_d * c * ρ)
    return Kn
end

function periodic(x, V)
    return (mod.(-(6 .- (x)), 12) .- 6), V
end

function periodic(x)
    return (mod.(-(6 .- (x)), 12) .- 6)
end

periodic (generic function with 2 methods)

### Plotting

In [48]:
function plotTrajcecotries(trajs, timer, fieldContour, colors, markers, tit, divider = 30*10)
    xs = Vector{Observable{Vector{Float64}}}(undef, size(trajs))
    ys = Vector{Observable{Vector{Float64}}}(undef, size(trajs))
    for i in 1:size(trajs)[1]
        xs[i] = Observable(trajs[i][1][1, :])
        ys[i] = Observable(trajs[i][1][2, :])
    end

    title = Observable(L"%$tit $t = %$(timer[1]) $")

    fig = GLMakie.Figure()
    display(fig)
    ax = Axis(fig[1, 1])

    GLMakie.contourf!(x, y, fieldContour, levels=20, color=:turbo)

    for i in 1:size(trajs)[1]
        GLMakie.scatter!(xs[i], ys[i], color=colors[i], marker=markers[i], strokewidth=2e-4)
    end

    Label(fig[1, 1, Top()], title, padding=(0, 0, 10, 0))
    Colorbar(fig[1, 2]; label=L"Ψ")

    steps = 10e100
    for i in 1:size(trajs)[1]
        if size(trajs[i])[1] < steps
            steps = size(trajs[i])[1]
        end
    end
    framerate = round(Int, steps / divider)
    for i in 1:framerate:steps
        sleep(1 / 30)
        title[] = L"%$tit $t = %$(timer[i]) $"
        for j in 1:size(trajs)[1]
            patching = size(trajs[j][1][1, :])[1] - size(trajs[j][i][1, :])[1]
            xs[j][] = vcat(trajs[j][i][1, :], (2 * xmax) .* ones(patching))
            ys[j][] = vcat(trajs[j][i][2, :], (2 * ymax) .* ones(patching))
        end
    end
end

function saveTrajcecotries(trajs, timer, fieldContour, colors, markers, tit, filename,divider = 30*10)
    xs = Vector{Observable{Vector{Float64}}}(undef, size(trajs))
    ys = Vector{Observable{Vector{Float64}}}(undef, size(trajs))
    cur_colors = theme_palette(:auto)
    for i in 1:size(trajs)[1]
        xs[i] = Observable(trajs[i][1][1, :])
        ys[i] = Observable(trajs[i][1][2, :])
    end
    title = Observable(L"$tit $t = %$(timer[1]) $")

    fig = GLMakie.Figure()
    display(fig)
    ax = Axis(fig[1, 1])

    GLMakie.contourf!(x, y, fieldContour, levels=20, color=:turbo)

    for i in 1:size(trajs)[1]
        GLMakie.scatter!(xs[i], ys[i], color=colors[i], marker=markers[i], strokewidth=2e-4)
    end

    Label(fig[1, 1, Top()], title, padding=(0, 0, 10, 0))
    Colorbar(fig[1, 2]; label=L"Ψ")

    steps = 10e100
    for i in 1:size(trajs)[1]
        if size(trajs[i])[1] < steps
            steps = size(trajs[i])[1]
        end
    end
    timestamps = 1:round(Int, steps / (divider)):steps

    record(fig, "$filename.mp4", timestamps; framerate=30) do i
        title[] = L"%$tit $t = %$(timer[i]) $"
        for j in 1:size(trajs)[1]
            patching = size(trajs[j][1][1, :])[1] - size(trajs[j][i][1, :])[1]
            xs[j][] = vcat(trajs[j][i][1, :], (2 * xmax) .* ones(patching))
            ys[j][] = vcat(trajs[j][i][2, :], (2 * ymax) .* ones(patching))
        end
    end
end

colors = [:red, :blue, :green];
markers = [:x, :circle, :circle];

# Cylinder Field

## Functions

In [49]:
function laplacianField_c!(Ψ, points)
    @views x = points[1, :]
    @views y = points[2, :]
    @fastmath r = @. sqrt(x .^ 2 + y .^ 2)
    @fastmath theta = @. atan(y, x)
    Ψ[:] = @fastmath @. U0 * (1 - (R / r) .^ 2) * r * sin(theta)
    nothing
end

function vFluid_c!(U, points)
    @views x = points[1, :]
    @views y = points[2, :]
    @fastmath r = @. sqrt(x .^ 2 + y .^ 2)
    @fastmath theta = atan.(y, x)

    @fastmath u_r = @. U0 * (1 - (R^2 / r^2)) * cos(theta)
    @fastmath u_th = @. -U0 * (1 + (R^2 / r^2)) * sin(theta)

    @fastmath u = @. u_r * cos(theta) - u_th * sin(theta)
    @fastmath v = @. u_r * sin(theta) + u_th * cos(theta)

    U[:] = hcat(u, v)'
    nothing
end

function gradVFluid_c!(u_x, u_y, v_x, v_y, points)
    @views x = points[1, :]
    @views y = points[2, :]

    @fastmath u_x[:] = @. 2 * R^2 * U0 * x * (x^2 - 3 * y^2) / ((x^2 + y^2)^3)
    @fastmath u_y[:] = @. -2 * R^2 * U0 * y * (y^2 - 3 * x^2) / ((x^2 + y^2)^3)
    @fastmath v_x[:] = u_y   # Due to potential flow
    @fastmath v_y[:] = -u_x  # Due to continuity
    nothing
end

function TempField!(T, coord)
    @views x = coord[1, :]
    @views y = coord[2, :]
    T[:] = zeros(size(x))
    for i in 1:size(x)[1]
        r = sqrt(x[i]^2 + y[i]^2)
        if r < R
            T[i] = 900
        elseif r < Rtr
            T[i] = 900 * (1 - (r - R) / (Rtr - R)) + 0.75 * 900 * (r - R) / (Rtr - R)
        else
            T[i] = 0.75 * 900
        end
    end
    nothing
end

function laplacianField_c(points)
    @views x = points[1, :]
    @views y = points[2, :]
    @fastmath r = @. sqrt(x .^ 2 + y .^ 2)
    @fastmath theta = @. atan(y, x)
    return @fastmath @. U0 * (1 - (R / r) .^ 2) * r * sin(theta)
end


function vFluid_c(points)
    @views x = points[1, :]
    @views y = points[2, :]
    @fastmath r = @. sqrt(x .^ 2 + y .^ 2)
    @fastmath theta = -atan.(y, x)

    @fastmath u_r = @. U0 * (1 - (R^2 / r^2)) * cos(-theta)
    @fastmath u_th = @. -U0 * (1 + (R^2 / r^2)) * sin(-theta)

    @fastmath u = @. u_r * cos(-theta) - u_th * sin(-theta)
    @fastmath v = @. u_r * sin(-theta) + u_th * cos(-theta)

    U = hcat(u, v)'
    return U
end

function TempField(coord)
    x = coord[1, :]
    y = coord[2, :]
    T = zeros(size(x))
    for i in 1:size(x)[1]
        r = sqrt(x[i]^2 + y[i]^2)
        if r < R
            T[i] = 900
        elseif r < Rtr
            T[i] = 900 * (1 - (r - R) / (Rtr - R)) + 0.75 * 900 * (r - R) / (Rtr - R)
        else
            T[i] = 0.75 * 900
        end
    end
    return T
end

TempField (generic function with 1 method)

In [50]:
function Per_Colisions(coord, V)
    global D_d
    global T_d
    global M_d
    @views x = coord[1, :]
    @views y = coord[2, :]
    idxs = []
    for i in 1:length(x)
        if x[i]^2 + y[i]^2 < R^2
            u = V[1, i]
            v = V[2, i]

            # Solve quadratic for collision point
            a = u^2 + v^2
            b = -2(x[i] * u + v * y[i])
            c = x[i]^2 + y[i]^2 - R^2
            Δ = b^2 - 4 * a * c

            dt = (-b + sqrt(Δ)) / (2a)
            x_col = x[i] - u * dt
            y_col = y[i] - v * dt

            # calculate Tangetial and radial Velocity
            θ = atan(y_col, x_col)   # angle of collision point
            V_rad = (u * cos(-θ) - v * sin(-θ))
            V_tan = (u * sin(-θ) + v * cos(-θ))

            # Flip Velocity due to collision
            V_rad = -V_rad

            # Convert to cartesian
            u_col = (V_rad * cos(θ) - V_tan * sin(θ))
            v_col = (V_rad * sin(θ) + V_tan * cos(θ))

            # Calculate new coordinates tore results
            coord[1, i] = x_col + u_col * dt
            coord[2, i] = y_col + v_col * dt
            V[1, i] = u_col
            V[2, i] = v_col
        else
            if (x[i] > xmax) | (x[i] < -xmax)
                push!(idxs, i)
            elseif (y[i] > ymax) | (y[i] < -ymax)
                push!(idxs, i)
            end
        end
    end

    coord = coord[:, setdiff(1:end, idxs)]
    V = V[:, setdiff(1:end, idxs)]

    D_d = D_d[setdiff(1:end, idxs)]
    T_d = T_d[setdiff(1:end, idxs)]
    M_d = M_d[setdiff(1:end, idxs)]

    return coord, V
end

Per_Colisions (generic function with 1 method)

## Plot Fields

### Flow Fields

In [51]:
Num_pts = 1000
y = LinRange(xmin, xmax, Num_pts)
x = LinRange(ymin, ymax, Num_pts)
points = zeros(Num_pts, Num_pts, 2)
for i in 1:Num_pts
    for j in 1:Num_pts
        if x[i]^2 + y[j]^2 > R^2
            points[i, j, 1] = x[i]
            points[i, j, 2] = y[j]
        end
    end
end
points = reshape(points, (Num_pts^2, 2))';

In [52]:
PSI_field_c = laplacianField_c(points);
PSI_field_c = reshape(PSI_field_c, (Num_pts, Num_pts));

isoPoints_c = isoPsi(1, 1e-3, PSI_field_c, x, y, Num_pts);

P_field_c = Pressure(points, vFluid_c)
P_field_c = reshape(P_field_c, (Num_pts, Num_pts))

U_c = vFluid_c(points);
u_field_c = reshape(U_c[1, :], (Num_pts, Num_pts));
v_field_c = reshape(U_c[2, :], (Num_pts, Num_pts));

In [53]:
# fig = Plots.contourf(x, y, [P_field_c', PSI_field_c', u_field_c', v_field_c'],
#     levels=100, color=:turbo, layout=(2, 2),
#     title=[L"P_{st}" "Ψ" L"$u$" L"$v$"],
#     size=(1200, 1000))
# savefig("Cylinder/Field.png")

### Temperature

In [54]:
y = LinRange(xmin, xmax, Num_pts)
x = LinRange(ymin, ymax, Num_pts)
points = zeros(Num_pts, Num_pts, 2)
for i in 1:Num_pts
    for j in 1:Num_pts
        points[i, j, 1] = x[i]
        points[i, j, 2] = y[j]
    end
end
points = reshape(points, (Num_pts^2, 2))';

In [55]:
T_field_c = TempField(points);
T_field_c = reshape(T_field_c, (Num_pts, Num_pts));

In [56]:
# fig = Plots.contourf(x, y, T_field_c', levels=20, color=:plasma, aspect_ratio=:equal, title=L"T", size=(800, 820))
# display(fig)
# savefig("Cylinder/Temperature_Field.png")

## Fluid Trajectory

In [57]:
p_num = 50
de = 0;
particleXs = LinRange(xmin + de, xmax - de, p_num);
particleYs = LinRange(ymin + de, ymax - de, p_num);
points = meshgrid(particleXs, particleYs);
points = reshape(points, (p_num^2, 2))';

for i in 1:p_num^2
    if points[1, i]^2 + points[2, i]^2 < R^2
        points[1, i] = -xmax
        points[2, i] = (i - ymax) % (2 * ymax) - (ymax - de)
    end
end
V0 = vFluid_c(points);

dt = 1e-3
t0 = 0
tend = 2 * (xmax - xmin) / U0
trajFlu = RungeKutta(vFluid_c, t0, tend, dt, points, periodic);
timer = t0:dt:tend;

In [58]:
# plotTrajcecotries([trajFlu], timer, PSI_field_c, colors, markers, "Fluid Trajectories")
# saveTrajcecotries([trajFlu], timer, PSI_field_c, colors, markers, "Fluid Trajectories", "Cylinder/Fluid_Streamline_Animation")

## Particle Trajectory

In [59]:
@fastmath function dvdtP_c!(F, X, V)
    vDims = size(X)

    Vfluid = Matrix{Float64}(undef, vDims)
    undisturbed = Matrix{Float64}(undef, vDims) #zeros(vDims)
    vMass = Matrix{Float64}(undef, vDims)
    ssDrag = Matrix{Float64}(undef, vDims) #zeros(vDims)
    brown = Matrix{Float64}(undef, vDims)

    vFluid_c!(Vfluid, X)

    u_x = Array{Float64}(undef, vDims[2])
    u_y = Array{Float64}(undef, vDims[2])
    v_x = Array{Float64}(undef, vDims[2])
    v_y = Array{Float64}(undef, vDims[2])

    gradVFluid_c!(u_x, u_y, v_x, v_y, X)

    @fastmath lift = @. (1 - ρ / ρ_d) * G

    τν = @. (ρ_d * (D_d^2) / (18 * μ))
    @fastmath @inbounds ssDrag[1, :] = @. (1 / τν) * (Vfluid-V)[1, :]
    @fastmath @inbounds ssDrag[2, :] = @. (1 / τν) * (Vfluid-V)[2, :]

    @fastmath @inbounds undisturbed[1, :] = @. (ρ / ρ_d) * (Vfluid[1, :] * u_x + Vfluid[2, :] * u_y)
    @fastmath @inbounds undisturbed[2, :] = @. (ρ / ρ_d) * (Vfluid[1, :] * v_x + Vfluid[2, :] * v_y)

    @fastmath @inbounds vMass[1, :] = @. 0.5 * (ρ / ρ_d) * (V[1, :] * u_x + V[2, :] * u_y)
    @fastmath @inbounds vMass[2, :] = @. 0.5 * (ρ / ρ_d) * (V[1, :] * v_x + V[2, :] * v_y)

    Brownian!(brown, X, V)

    F[:] = @. dt * (undisturbed + vMass + lift + ssDrag) / (1 + 0.5 * ρ / ρ_d) + dt* brown
    nothing
end

function Brownian!(F, coord, V)
    d = Normal(0, 1)
    Gaussian = rand(d, size(coord))

    T_f = TempField(coord)
    Kn = Knudsen(coord, T_f)
    C_c = @. 1 / (1 + Kn * (2.49 + 0.84 * exp(-1.74 / Kn)))
    S = @. 216 * (μ / ρ) * k_b * T_f / ((π^2) * ρ_d * (D_d^5) * (ρ_d / ρ)^2 * C_c)

    foo = hcat(sqrt.(π * S / dt), sqrt.(π * S / dt))'
    f_t = @. Gaussian * foo
    # B = @. C_c / (3 * π * μ * D_d)

    F[:] = f_t #- (1 / B) * V
    nothing
end

Brownian! (generic function with 1 method)

In [60]:
global Du = 2.42e-5;
global Hum_rel = 2 / 100;
global Patm = 1e5;
global hlatent = 2.260e6
global Cp_d = 4.18e3
global k = 0.028;

function evaporation(pts, Vel)
    global D_d
    global M_d
    global T_d

    global Devap
    global Tevap
    global Mevap

    ## Get Fluid Temeperature and Velocity
    T_f = Array{Float64}(undef, size(pts)[2])
    VFluid = Matrix{Float64}(undef, size(pts))

    TempField!(T_f, pts)
    vFluid_c!(VFluid, pts)

    ## Calculate Humidity
    #Psat = @. 610.7 * exp10(7.5 * (T_d - 273.15) / (T_d - 35.85))
    @fastmath Psat = @. 610.94 * exp(17.625 * (T_d - 273.15) / (T_d - 273.15 + 243.04))

    @fastmath wAs = @. (18.01528 / 28.96) * (Psat / Patm)
    @fastmath wAinf = wAs * Hum_rel

    ## Calculate Flow Numbers
    @fastmath magDv = @. sqrt((VFluid[1, :] - Vel[1, :])^2 + (VFluid[2, :] - Vel[2, :])^2)
    @fastmath Reyn_r = @. magDv * D_d * ρ / μ
    @fastmath Schmidt = μ / (ρ * Du)
    @fastmath Sherwood = @. 2 + 0.6 * (Reyn_r^0.5) * (Schmidt^0.33)
    @fastmath Prandtl = Cp_d * (μ / k)
    @fastmath Nusselt = @. 2 + 0.4 * (Reyn_r^0.5) * (Prandtl^0.33)

    ## Calculate Heat and Mass Transfer Coeeffs
    @fastmath Qdot = @. (π * D_d) * (Nusselt * k) * (T_f - T_d)
    @fastmath mdot = @. Sherwood * π * D_d * ρ * Du * (wAinf - wAs)
    @fastmath λ = @. 4 * Sherwood * (ρ / ρ_d) * Du * (wAs - wAinf)

    # Check if the Particle still exists and calculate new Properties
    idxs = []
    for i in 1:size(D_d)[1]
        if (M_d[i] + mdot[i] * dt  < 0) | (D_d[i]^2 - λ[i] * dt < 0)
            push!(idxs, i)
        else
            ## Euler Integration of Equations
            mass  = D_d[i]^3 * ρ_d *  π / 6
            D_d[i] = sqrt(D_d[i]^2 - λ[i] * dt)                                     # New Diam
            T_d[i] = T_d[i] + (Qdot[i] + hlatent * mdot[i]) * dt / (mass * Cp_d)    # New Temp
            M_d[i] = M_d[i] + mdot[i] * dt                                          # New Mass
            
            ## Parcel Location
            N = ceil(Int, (pts[1, i] - xmin) / Dx)
            M = ceil(Int, (pts[2, i] - ymin) / Dy)
            Mevap[N, M] = Mevap[N, M] - mdot[i] * dt
            Devap[N, M] = Devap[N, M] + D_d[i]
            Tevap[N, M] = Tevap[N, M] + T_d[i] 
            C[N, M] = C[N, M] + 1
        end
    end
    ## Remove evaporated Particles
    pts = pts[:, setdiff(1:end, idxs)]
    Vel = Vel[:, setdiff(1:end, idxs)]
    D_d = D_d[setdiff(1:end, idxs)]
    T_d = T_d[setdiff(1:end, idxs)]
    M_d = M_d[setdiff(1:end, idxs)]

    return pts, Vel
end

evaporation (generic function with 1 method)

In [61]:
global μ = 1.813e-5
global ρ = 1.2

global ρ_d = 1000
global d_p = invStokes(0.1)
global V_d = (π / 6) * (d_p^3)
global m_P = V_d * ρ_d
global m_F = V_d * ρ

St = round(Stokes(d_p),sigdigits = 3)
println("Stokes Number is $St\ndiameter: $d_p\ndensity: $ρ_d")
Rmaxie = round((ρ / (0.5 * ρ + ρ_d)),sigdigits=3)
println("R is $Rmaxie")

Stokes Number is 0.1
diameter: 4.039430652950982e-5
density: 1000
R is 0.0012


### Particle Grid

In [62]:
p_num = 15000
pxs = ones(p_num) * xmin
pys = LinRange(ymin, ymax, p_num)
points = hcat(pxs, pys)'
V0 = vFluid_c(points);

### Initialize Variables

In [63]:
global dt = 1e-4;
global Dx = U0 * dt * 5 ;
global Dy = U0 * dt * 5 ;
global Nx = ceil(Int, 2 * xmax / Dx);
global Ny = ceil(Int, 2 * ymax / Dy)

800

In [76]:
# Particles
global D_d = d_p * ones(p_num);
global T_d = 300 * ones(p_num);
global M_d = @. (d_p^3) * ρ_d *  (π / 6) * ones(p_num);

### Integrate

In [77]:
t0 = 0;
tend = 2 * (xmax - xmin) / U0
timer = t0:dt:tend;
println("Simulation from t0=$t0 to tend=$tend with dt=$dt")
println("Steps: $(round(Int,(tend-t0)/dt))")

Simulation from t0=0 to tend=0.8 with dt=0.0001
Steps: 8000


## Trajectories

In [78]:
trajP_noEvap = RK4systems(dxdtP!, dvdtP_c!, t0, tend, dt, points, V0, [Per_Colisions]);

In [ ]:
saveTrajcecotries([trajP_noEvap], timer, PSI_field_c, colors, markers,
            L"Fluid Trajectories with Evaporation ($ R: %$(Rmaxie): St:%$(St) RH:%$(Hum_rel) $)", "Cylinder/Evaporation_R_$(Rmaxie)_St_$(St)_RH_$(Hum_rel)")

In [88]:
    global μ = 1.813e-5
    global ρ = 1.2

    global ρ_d = 0.6
    global d_p = invStokes(1)
    global V_d = (π / 6) * (d_p^3)
    global m_P = V_d * ρ_d
    global m_F = V_d * ρ

    # Particles
    global D_d = d_p * ones(p_num);
    global T_d = 300 * ones(p_num);
    global M_d = @. (d_p^3) * ρ_d * (π / 6) * ones(p_num);

    St = round(Stokes(d_p), sigdigits=3)
    println("Stokes Number is $St\ndiameter: $d_p\ndensity: $ρ_d")
    Rmaxie = round((ρ / (0.5 * ρ + ρ_d)), sigdigits=3)
    println("R is $Rmaxie")

Stokes Number is 1.0
diameter: 0.0052148825490129684
density: 0.6
R is 1.0


In [90]:
for number in [0.001,0.5,10]
    for rho in [0.01, 1000, 0.6]
        global μ = 1.813e-5
        global ρ = 1.2

        global ρ_d = 1000
        global d_p = invStokes(number)
        global V_d = (π / 6) * (d_p^3)
        global m_P = V_d * ρ_d
        global m_F = V_d * ρ

        # Particles
        global D_d = d_p * ones(p_num);
        global T_d = 300 * ones(p_num);
        global M_d = @. (d_p^3) * ρ_d * (π / 6) * ones(p_num);

        St = round(Stokes(d_p), sigdigits=3)
        println("Stokes Number is $St\ndiameter: $d_p\ndensity: $ρ_d")
        Rmaxie = round((ρ / (0.5 * ρ + ρ_d)), sigdigits=3)
        println("R is $Rmaxie")

        trajP_noEvap = RK4systems(dxdtP!, dvdtP_c!, t0, tend, dt, points, V0, [Per_Colisions]);
        saveTrajcecotries([trajP_noEvap], timer, PSI_field_c, colors, markers,
            L"Fluid Trajectories with Evaporation ($ R: %$(Rmaxie): St:%$(St) RH:%$(Hum_rel) $)", "Cylinder/Trajectories_R_$(Rmaxie)_St_$(St)_RH_$(Hum_rel)")
    end
end

Stokes Number is 0.001
diameter: 4.039430652950983e-6
density: 1000
R is 0.0012


Stokes Number is 0.001
diameter: 4.039430652950983e-6
density: 1000
R is 0.0012


## Evaporation

In [ ]:
# Parcels
global Mevap = zeros(Nx, Ny);
global Devap = zeros(Nx, Ny);
global Tevap = zeros(Nx, Ny);
global C = 1e-10 * ones(Nx, Ny);

In [66]:
trajP = RK4systems(dxdtP!, dvdtP_c!, t0, tend, dt, points, V0, [Per_Colisions,evaporation]);
# trajFlu = RungeKutta(vFluid_c, t0, tend, dt, points, periodic);

485-element Vector{Matrix{Float64}}:
 [-2.0 -2.0 … -2.0 -2.0; -2.0 -1.9997333155543702 … 1.9997333155543702 2.0]
 [-1.999000004095607 -1.9990000082654356 … -1.9990000082673534 -1.9990000041009315; -1.9997646184235291 -1.9994979381430442 … 1.9994978408765445 1.9997645211423398]
 [-1.9980000078408713 -1.9980000161837055 … -1.998000016185692 -1.9980000078459177; -1.9997960176057306 -1.9995293414943598 … 1.9995289549905346 1.9997956310682359]
 [-1.9970000109270145 -1.997000023439056 … -1.9970000234460232 -1.9970000109228276; -1.9998275114914852 -1.9995608395554685 … 1.9995599756391078 1.9998266475276305]
 [-1.996000013044887 -1.9960000297282117 … -1.9960000297488674 -1.996000013039364; -1.9998590985043614 -1.9995924307434823 … 1.9995909049726326 1.999857572674867]
 [-1.9950000138898383 -1.995000034748045 … -1.9950000347840884 -1.9950000138951984; -1.999890777102308 -1.9996241135150654 … 1.9996217451080007 1.9998884086148616]
 [-1.9940000131630988 -1.994000038203237 … -1.9940000382569416 -1

In [73]:
plotTrajcecotries([trajP], timer, PSI_field_c, colors, markers, 
            L"Fluid Trajectories with Evaporation ($ R: %$(Rmaxie): St:%$(St) RH:%$(Hum_rel) $)")

In [67]:
saveTrajcecotries([trajP], timer, PSI_field_c, colors, markers,
            L"Fluid Trajectories with Evaporation ($ R: %$(Rmaxie): St:%$(St) RH:%$(Hum_rel) $)", "Cylinder/Evaporation_R_$(Rmaxie)_St_$(St)_RH_$(Hum_rel)")

"Cylinder/Evaporation_R_0.0012_St_0.1_RH_0.02.mp4"

# Visualize Parcels

In [68]:
for j in 1:Ny
    for i in 1:Nx
        Devap[i, j] = Devap[i, j] / C[i, j]
        Tevap[i, j] = Tevap[i, j] / C[i, j]
        # Mevap[i, j] = Mevap[i, j] / C[i, j]
        if C[i, j] == 0
            Tevap[i, j] = 300
        end
    end
end
x2 = LinRange(xmin, xmax, Nx)
y2 = LinRange(ymin, ymax, Ny)

800-element LinRange{Float64, Int64}:
 -2.0,-1.99499,-1.98999,-1.98498,…,1.97997,1.98498,1.98999,1.99499,2.0

In [69]:
fig = GLMakie.contourf(x2, y2, Devap, levels=20, color=:plasma, aspect_ratio=:equal, size=(800, 820))
print("saving...")
save("Cylinder/AvDiameter_R_$(Rmaxie)_St_$(St)_RH_$(Hum_rel).png", fig)

saving...

In [70]:
fig = GLMakie.contourf(x2, y2, Mevap, levels=20, color=:plasma, aspect_ratio=:equal, size=(800, 820))
print("saving...")
save("Cylinder/AvMass_R_$(Rmaxie)_St_$(St)_RH_$(Hum_rel).png", fig)

saving...

In [71]:
fig = GLMakie.contourf(x2, y2, Tevap, levels=20, color=:plasma, aspect_ratio=:equal, size=(800, 820))
print("saving...")
save("Cylinder/AvTemperature_$(Rmaxie)_St_$(St)_RH_$(Hum_rel).png", fig)

saving...